-------------

# Imports

In [ ]:
#%%capture
#!rm -r Foo_env
#!git clone https://github.com/username/Foo_env.git
#!pip install -e Foo_env

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Senior_Project/Snake_Plus_Plus/Snake_Base_Game/')
!pwd


/content
/content/drive/MyDrive/Colab Notebooks/Senior_Project/Snake_Plus_Plus/Snake_Base_Game


In [ ]:
#!ls drive/MyDrive/Colab\ Notebooks/MY_STUFF/Snake_Base/
#!pip install -e Foo_env

In [ ]:
#!cd drive/MyDrive/Colab\ Notebooks/MY_STUFF/Snake_Base/

In [ ]:
#!pip install -e snake

In [ ]:
!pip install keras-rl2==1.0.4
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!unzip /content/gym-foo.zip
#!pip install -e /content/

In [ ]:
#import os

#def restart_runtime():
#  os.kill(os.getpid(), 9)

#restart_runtime()

In [ ]:

from PIL import Image  # To transform the image in the Processor
import numpy as np
import gym

# Convolutional Backbone Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam

# Keras-RL
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint


In [ ]:
!pip install -e snake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/Senior_Project/Snake_Plus_Plus/Snake_Base_Game/snake
  Preparing metadata (setup.py) ... done
  Attempting uninstall: gymsnake
    Found existing installation: gymsnake 0.0.1
    Can't uninstall 'gymsnake'. No files were found to uninstall.
  Running setup.py develop for gymsnake


In [ ]:
os.environ['SDL_VIDEODRIVER']='dummy'
env = gym.make("snake:snake-v0")
nb_actions = env.action_space.n
#nb_actions = env.action_space.env = gym.make("snake:snake-v0")


/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


-----
# Image Processing

In [ ]:
nb_actions

4

In [ ]:
IMG_SHAPE = (84, 84)
WINDOW_LENGTH = 4


In [ ]:
class ImageProcessor(Processor):
    def process_observation(self, observation):
        # First convert the numpy array to a PIL Image
        img = Image.fromarray(observation)
        # Then resize the image
        img = img.resize(IMG_SHAPE)
        # And convert it to grayscale  (The L stands for luminance)
        img = img.convert("L")
        # Convert the image back to a numpy array and finally return the image
        img = np.array(img)
        return img.astype('uint8')  # saves storage in experience memory
    
    def process_state_batch(self, batch):

        # We divide the observations by 255 to compress it into the intervall [0, 1].
        # This supports the training of the network
        # We perform this operation here to save memory.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)


## Model Creation

**NOTE: Depending on your custom environment, this model will vary greatly, try reading papers that are solving similar problems to your own!**

In [ ]:
input_shape = (WINDOW_LENGTH, IMG_SHAPE[0], IMG_SHAPE[1])
input_shape

(4, 84, 84)

In [ ]:
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))

model.add(Convolution2D(32, (8, 8), strides=(4, 4),kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 activation (Activation)     (None, 20, 20, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 activation_1 (Activation)   (None, 9, 9, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 activation_2 (Activation)   (None, 7, 7, 64)          0

----
## Creating the Agent

In [ ]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)


In [ ]:
processor = ImageProcessor()


In [ ]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)


In [ ]:
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
              train_interval=4, delta_clip=1)

In [ ]:
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])


In [ ]:
weights_filename = 'test_dqn_snake_weights.h5f'
checkpoint_weights_filename = 'test_dqn_' + "snake" + '_weights_{step}.h5f'
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_weights_filename, interval=100000)


In [ ]:
dqn.fit(env, nb_steps=1500000, callbacks=[checkpoint_callback], log_interval=100000, visualize=False)

# After training is done, we save the final weights one more time.
dqn.save_weights(weights_filename, overwrite=True)



Training for 1500000 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automa

    22/100000 [..............................] - ETA: 12:18 - reward: 0.0000e+00

/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


100000/100000 [==============================] - 585s 6ms/step - reward: -0.0211
2317 episodes - episode_reward: -0.912 [-1.000, 2.000] - loss: 0.003 - mae: 0.074 - mean_q: 0.089 - mean_eps: 0.932 - score: 0.069

Interval 2 (100000 steps performed)
100000/100000 [==============================] - 847s 8ms/step - reward: -0.0206
2338 episodes - episode_reward: -0.881 [-1.000, 1.000] - loss: 0.002 - mae: 0.305 - mean_q: 0.436 - mean_eps: 0.865 - score: 0.086

Interval 3 (200000 steps performed)
100000/100000 [==============================] - 915s 9ms/step - reward: -0.0128
1893 episodes - episode_reward: -0.678 [-1.000, 2.000] - loss: 0.004 - mae: 0.700 - mean_q: 0.971 - mean_eps: 0.775 - score: 0.239

Interval 4 (300000 steps performed)
100000/100000 [==============================] - 1016s 10ms/step - reward: -0.0066
1796 episodes - episode_reward: -0.367 [-1.000, 3.000] - loss: 0.006 - mae: 1.042 - mean_q: 1.443 - mean_eps: 0.685 - score: 0.401

Interval 5 (400000 steps performed)
10

In [ ]:
# Load the weights
model.load_weights("test_dqn_snake_weights.h5f")


memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1, value_min=.1, value_test=.05,
                              nb_steps=100000)

processor = ImageProcessor()

# Initialize the DQNAgent with the new model and updated policy and compile it
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000)
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])


In [ ]:
env.sleep = 2

In [ ]:
dqn.test(env, nb_episodes=30, visualize=True)

Testing for 30 episodes ...


/usr/local/lib/python3.8/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:280: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


Episode 1: reward: 10.000, steps: 149
Episode 2: reward: 5.000, steps: 110
Episode 3: reward: 8.000, steps: 191
Episode 4: reward: 10.000, steps: 198
Episode 5: reward: 11.000, steps: 183
Episode 6: reward: 12.000, steps: 257
Episode 7: reward: 8.000, steps: 146
Episode 8: reward: 10.000, steps: 185
Episode 9: reward: 8.000, steps: 155
Episode 10: reward: 8.000, steps: 143
Episode 11: reward: 16.000, steps: 324
Episode 12: reward: 5.000, steps: 70
Episode 13: reward: 14.000, steps: 258
Episode 14: reward: 7.000, steps: 148
Episode 15: reward: 14.000, steps: 256
Episode 16: reward: 13.000, steps: 224
Episode 17: reward: 9.000, steps: 184
Episode 18: reward: 5.000, steps: 95
Episode 19: reward: 6.000, steps: 135
Episode 20: reward: 12.000, steps: 196
Episode 21: reward: 10.000, steps: 229
Episode 22: reward: 13.000, steps: 205
Episode 23: reward: 5.000, steps: 125
Episode 24: reward: 14.000, steps: 294
Episode 25: reward: 11.000, steps: 194
Episode 26: reward: 9.000, steps: 194
Episode 2